<a href="https://colab.research.google.com/github/mzndnes/Deep-Learning/blob/main/Copy_of_real_time_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# function to convert the JavaScript object into an OpenCV image
from base64 import b64decode, b64encode
import numpy as np
import cv2
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img



In [ ]:
# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
import PIL
import io
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [ ]:
## class
import numpy as np
from tensorflow.keras.models import load_model
import time 

class RealTimeFace:
  def __init__(self):
    path='./drive/MyDrive/'
    self.fl_unum=path+'data/unum.npy'
    self.fl_usr_cat=path+'data/user_cat.npy'
    self.fl_img_embs=path+'data/image_embs.npy'
    self.tem_face=[]
    self.model=load_model('./drive/My Drive/data/facenet_keras.h5',compile=False)
    try:
      self.unum=np.load(self.fl_unum)
      self.usr_cat=np.load(self.fl_usr_cat)
      self.img_embs=np.load(self.fl_img_embs)
      self.id=self.unum[0]

      
    except:
      print('No users exists')
      self.unum=[]
      self.usr_cat=[]
      self.img_embs=[]
      self.id=0
    

  def create_embeddings(self,img):
      img = np.expand_dims(img, axis=0)
      return self.model.predict(img)[0]

  def register(self,name):  
   
    face=self.detect_face(1)
        
    self.unum.append(self.id+1)
    np.save(self.fl_unum,np.array(self.unum))
    np.save(self.fl_usr_cat,np.array(self.usr_cat))
    np.save(self.fl_img_embs,np.array(self.img_embs))  
  
  def recognize_face(self,emb):
    
    eu_dist=[]

    for em in self.img_embs:
      eu_dist.append(np.linalg.norm(emb-em))
      

    eu_dist=np.array(eu_dist)
    eu_exp=np.exp(eu_dist)
    sum_exp=np.sum(eu_exp)
    eu_softmax=eu_exp/sum_exp

    print(eu_softmax)
    
    
    idx=np.argsort(eu_softmax)
    for i in idx:
      if eu_softmax[i]<0.5:
        return self.usr_cat[i][0],str(self.usr_cat[i][0])
      else:
        return 'Not', 'Registered'


        
  def detect_face(self,r=0):
   
    path='./drive/My Drive/images/realtime_tr/'

    face_detector = cv2.CascadeClassifier('./drive/My Drive/data/haarcascade_frontalface_default.xml')
    
    
    video_stream()
    # label for video
    label_html = 'Capturing...'
    # initialze bounding box to empty
    bbox = ''
    count = 0 
    while True:
      js_reply = video_frame(label_html, bbox)
      if not js_reply:
        break

      # convert JS response to OpenCV Image
      img = js_to_image(js_reply["img"])
      
      
      # create transparent overlay for bounding box
      bbox_array = np.zeros([480,640,4], dtype=np.uint8)

      # grayscale image for face detection
      gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

      # get face region coordinates
      faces = face_cascade.detectMultiScale(gray)
      # get face bounding box for overlay

      for (x,y,w,h) in faces:
        bbox_array = cv2.rectangle(bbox_array,(x,y),(x+w,y+h),(255,0,0),2)
        #cv2.putText(bbox_array, "hi", (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
      
      if r==0 and len(faces)==1:
        (x,y,w,h)=faces[0]
        face=img[y:y+w,x:x+h]
        face=cv2.resize(face,(160,160))
        emb = self.create_embeddings(face)
        nm,id=self.recognize_face(emb)
        cv2.putText(bbox_array, nm+id, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
        time.sleep(10)
        

      if len(faces)==1 and count<3 and r==1:
        (x,y,w,h)=faces[0]
        face=img[y:y+w,x:x+h]
        face=cv2.resize(face,(160,160))

        self.tem_face.append(face)
    
    
        tem_usr_cat=[]
        fl=name+'-'+str(count)+'.jpg'
        tem_usr_cat.append(fl)
        tem_usr_cat.append(self.id)
        self.usr_cat.append(tem_usr_cat)
          
        
        emb = self.create_embeddings(face)
        self.img_embs.append(emb)
        
        cv2.imwrite(path+fl,face)
        count+=1 

      if count==3:
        print('Please Close Camera')    
        count=4

      bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
      # convert overlay of bbox into bytes
      bbox_bytes = bbox_to_bytes(bbox_array)
      # update bbox so next frame gets new overlay
      bbox = bbox_bytes
        


In [ ]:
import numpy as np
# def main():
obj=RealTimeFace()

choice = input("Are you a Registered User: Y/N ")
if choice.upper() == "N":
    name = input("Enter your Name and be ready infront of camera: ")
    obj.register(name)
    print("Registration Successful and your id is",self.id)
  


In [ ]:
ch = input("Would you like to be recognized: Y/N ")
obj=RealTimeFace()
if ch.upper() == 'Y':
    obj.detect_face()
    


Would you like to be recognized: Y/N y


<IPython.core.display.Javascript object>

[9.1020161e-07 5.1602763e-07 9.9999857e-01]
[6.0042006e-01 3.9957443e-01 5.5140081e-06]
